# Philip Carr
# CS/CNS/EE_156a_Homework_5_Code_Part_2 (Jupyter Notebook)

Code for The Logistic Regression Algorithm (Problems 8 and 9)

In [14]:
import random as rn
import numpy as np
from matplotlib import pyplot as plt

In [15]:
def sign(x):
    '''
    Return the sign of a number (1 if positive, 0 if 0, or -1 if
    negative).
    
    Return type: int
    '''
    if x >= 0:
        return 1
    elif x == 0:
        return 0
    else:
        return -1

In [16]:
def target_function_line_value(x_point, y_point, m, b):
    '''
    Return the evaluation of the target function (line with slope m
    and y-intercept b) for a given point (1, x_point, y_point) and
    sign_orientation: determines whether points have value of 1 or -1
    when above the target function line with this value being either
    1 or -1 respectively.
    
    Return type: int
    '''
    y_line = m * x_point + b
    if y_point >= y_line:
        return 1
    else:
        return -1

In [17]:
def get_target_function_line():
    '''
    Return a randomly generated target function line's slope (m) and
    y-intercept (b).
    
    Return type: tuple of floats
    '''
    x1 = rn.uniform(-1, 1)
    y1 = rn.uniform(-1, 1)
    x2 = rn.uniform(-1, 1)
    y2 = rn.uniform(-1, 1)
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return m, b

In [18]:
def get_target_function():
    '''
    Return a randomly generated target function line.
    
    Return type: function that takes two floats and returns an int
    (instance of the target_function_line_value function).
    '''
    m, b = get_target_function_line()
    return m, b, lambda x_point, y_point: \
        target_function_line_value(x_point, y_point, m, b)

In [19]:
def get_random_points(N):
    '''
    Return a list of randomly generated points within
    the region [-1, 1] x [-1, 1].
    
    Return type: list of points (each point is a list)
    '''
    random_points = []
    for n in range(N):
        x0 = 1.0 # artificial coordinate
        x1 = rn.uniform(-1, 1)
        x2 = rn.uniform(-1, 1)
        random_points.append([x0, x1, x2])
    return random_points

In [20]:
def get_point_values(points, target_function):
    '''
    Return the values corresponding to the (list of) points given.
    The returned values are function evaluations of the target
    function target_function of the given points.
    
    Return type: list of ints
    '''
    point_values = []
    for i in range(len(points)):
        value = target_function(points[i][1], points[i][2])
        point_values.append(value)
    return point_values

In [21]:
class LogReg:
    '''
    This class represents the LogReg (Logistic Regression Algorithm).
    This class contains the weights, as well as methods for running
    the Logistic Regression Algorithm.
    '''
    def __init__(self, n=2):
        '''
        Initialize the weights of the LogReg using the given
        dimension n of the points to work with in R^n space.
        
        Return type: class (LogReg)
        '''
        self.weights = [0] * (n + 1)
    
    def __repr__(self):
        '''
        Print the weights of the LogReg.
        
        Return type: string
        '''
        print("LogReg weights:", self.weights)
    
    def get_weights(self):
        '''
        Return the weights of the LogReg.
        
        Return type: list of floats
        '''
        return self.weights
    
    def updated_weights(self, point, value, eta):
        '''
        Update the LogReg weights using the gradient given a point
        and its corresponding target function value. Eta is the
        learning rate.
        
        Return type: list of floats
        '''
        new_weights = []
        gradE = [0] * len(self.weights)
        # Calculate the gradient for each point and update the
        # weights accordingly.
        denominator_exp_term = 0.0
        for i in range(len(point)):
            denominator_exp_term += \
                self.weights[i] * point[i]
        denominator_exp_term *= value
        for i in range(len(point)):
            gradE[i] = -1 * point[i] * value \
                       / (1 + np.exp(denominator_exp_term))
        for i in range(len(point)):
            new_weights.append(self.weights[i] - eta * gradE[i])
        return new_weights

    def get_weights_iteration_distance(self, current_weights,
                                       new_weights):
        '''
        Return the Euclidean distance between two weight vectors.
        '''
        sum_of_squares = 0.0
        for i in range(len(current_weights)):
            weight_distance = (new_weights[i] - current_weights[i])
            sum_of_squares += weight_distance * weight_distance
        return np.sqrt(sum_of_squares)
        
    def run(self, points, values, max_epochs=1000, eta=0.01,
            error_threshold=0.01):
        '''
        Return the number of iterations it takes to converge the
        LogReg to minimize classification error given (list of)
        points and (the list of) target function point values of the
        given points (values).
        
        Return type: int
        '''
        n_epochs = 0
        weights_iteration_distance = float("inf")
        N = len(points)
        while weights_iteration_distance >= error_threshold \
              and n_epochs < max_epochs:
            permutation_indices = np.random.permutation(N)
            old_weights = list(np.copy(np.array(self.weights)))
            for i in range(N):
                point = points[permutation_indices[i]]
                value = values[permutation_indices[i]]
            
                new_weights = self.updated_weights(point, value,
                                                   eta)
                self.weights = new_weights
            
            weights_iteration_distance = \
                self.get_weights_iteration_distance(old_weights,
                                                    new_weights)
            n_epochs += 1
        return n_epochs

In [22]:
def cross_entropy_error(trained_LR, N, t_function, points, values):
    '''
    Return the cross entropy error given a trained Logistic
    Regression Algorithm, the number of points used, the target
    function, the points, and the points' corresponding values.
    '''
    weights = trained_LR.get_weights()
    term_sum = 0
    for n in range(N):
        exp_term = 0
        for i in range(len(points[0])):
            try:
                exp_term += weights[i] * points[n][i]
            except:
                print(len(weights), len(points[n]))
                raise ValueError("list index out of range")
        exp_term *= -values[n]
        term_sum += np.log(1 + np.exp(exp_term))
    return term_sum / N

In [23]:
def get_E_out(trained_LR, N, t_function):
    '''
    Return the cross entropy error given a trained Logistic
    Regression Algorithm for out-of-sample data.
    '''
    points = get_random_points(N)
    values = get_point_values(points, t_function)
    
    return cross_entropy_error(trained_LR, N, t_function, points,
                               values)

In [24]:
def get_LogReg_error(N_in=100, N_out=1000):
    '''
    Return the number of iterations to converge and out-of-sample
    error of a run of the Logistic Regression Algorithm.
    '''
    training_points = get_random_points(N_in)
    m, b, t_function = get_target_function()
    training_values = get_point_values(training_points, t_function)
    LR = LogReg()
    n_epochs = LR.run(training_points, training_values)
    E_out = get_E_out(LR, N_out, t_function)
    return n_epochs, E_out

In [25]:
def LogReg_test(trials=1000, N_in=100, N_out=1000):
    '''
    Return the average number of iterations to converge and average
    out-of-sample error of a run of the Logistic Regression
    Algorithm over a given number of trials.
    '''
    n_epochs_sum = 0
    E_out_sum = 0
    for i in range(trials):
        n_epochs, E_out = get_LogReg_error(N_in=N_in, N_out=N_out)
        n_epochs_sum += n_epochs
        E_out_sum += E_out
    mean_n_epochs = n_epochs_sum / trials
    mean_E_out = E_out_sum / trials
    print("Mean E_out for N =", N_in, "for", trials, ":", mean_E_out)
    print("Mean number of epochs to converge for N =", N_in, "for",
          trials, "trials:", mean_n_epochs)

For Problems 8 and 9

In [26]:
LogReg_test()

Mean E_out for N = 100 for 1000 : 0.10264816397345258
Mean number of epochs to converge for N = 100 for 1000 trials: 343.102
